In [1]:
# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
get_ipython().magic('env CUDA_VISIBLE_DEVICES = ')
#%env CUDA_VISIBLE_DEVICES = 0
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave, imshow, imresize
import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.)
tf_sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options))
# now tensorflow will assume there not exist gpu
# use this tf_session following, and close in the end
#tf_sess.close()

import os
import os.path
import random
import time
import io
from datetime import datetime
import gc # garbage collector
import logging
get_ipython().magic('env CUDA_VISIBLE_DEVICES = 0')
from tensorboard_logger import configure, log_value
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.autograd import Variable
import torch
import torch_resnet

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
get_ipython().magic('matplotlib inline')
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')

env: CUDA_VISIBLE_DEVICES=
env: CUDA_VISIBLE_DEVICES=0


In [2]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/'
PRETRAINED_MODEL_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/models/resnet152-b121ed2d.pth'
LOG_DIR = DATASET_PATH + 'pytorch/cdiscount/logs_resnet'
LR_FILE_PATH = DATASET_PATH + 'pytorch/cdiscount/logs_resnet/lr_setting/resnet_lr_setting'
TRAIN_PATH = DATASET_PATH + 'Split/Train/'
#TRAIN_PATH = '/media/rs/FC6CDC6F6CDC25E4/resample_dataset2/'
VAL_PATH = DATASET_PATH + 'Split/Validation/'
TEST_PATH = DATASET_PATH + 'Test/'
CATEGORY_NAME_PATH = DATASET_PATH + 'category_names.csv'
CATEGORY_WEIGHT_PATH = DATASET_PATH + 'catogory_with_weight.csv'

# following is non-resampled, and only 10mil
LEVEL1_WEIGHT = [1.817, 1.528, 1.056, 2.174, 1.622, 2.531, 2.663, 1.103, 1.935, 1.937, 2.898, 1.279, 2.923, 1.095, 2.086, 3.0, 2.218, 3.0, 3.0, 2.123, 3.0, 1.038, 3.0, 3.0, 3.0, 3.0, 1.817, 1.448, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 1.955, 3.0, 1.946, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]

BATCH_SIZE = 128#256

IMAGE_WIDTH = 180
IMAGE_HEIGHT = 180
NUM_CLASS = 5270
LEVEL1_CLASS = 49
LEVEL2_CLASS = 483
# validation examples num: 2319624
# train examples num: 10051704
# total step: 157057
TOTAL_EXAMPLES = 12301740
VAL_EXAMPLES = 69588

NUM_EPOCHES = 12
EPOCHES_OVER = 8

INPUT_THREADS = 8

save_time_interval = 720
log_step_interval = 10

initial_learning_rate = 0.01#0.0004
stop_learning_rate = 0.0000001
momentum = 0.9
num_steps_per_train_epoch = int(TOTAL_EXAMPLES / (BATCH_SIZE)) + 1
num_steps_per_val_epoch = int(VAL_EXAMPLES / (BATCH_SIZE)) + 1

In [3]:
def set_logging(logger_name, logger_file_name):
    log = logging.getLogger(logger_name)
    log.setLevel(logging.DEBUG)

    # create formatter and add it to the handlers
    print_formatter = logging.Formatter('%(message)s')
    file_formatter = logging.Formatter('%(asctime)s - %(name)s_%(levelname)s: %(message)s')

    # create file handler which logs even debug messages
    fh = logging.FileHandler(logger_file_name, mode='w')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(file_formatter)
    log.addHandler(fh)
    # both output to console and file
    consoleHandler = logging.StreamHandler()
    consoleHandler.setFormatter(print_formatter)
    log.addHandler(consoleHandler)
    
    return log

In [4]:
log = set_logging('CDiscount', DATASET_PATH + 'pytorch/cdiscount/resnet_pytorch.log')
log.info('here is an info message.')

here is an info message.


In [5]:
class Saver(object):
    def __init__(self, log_dir, ckeckpoint_name, max_to_keep=5):
        super(Saver, self).__init__()
        self._log_dir = log_dir
        self._max_to_keep = max_to_keep
        self._ckeckpoint_container_file = os.path.join(self._log_dir, 'checkpoint')
        self._ckeckpoint_name = os.path.join(self._log_dir, ckeckpoint_name)
        self._best_ckeckpoint_name = os.path.join(self._log_dir, 'best_checkpoint')
        os.makedirs(self._log_dir, exist_ok=True)
        os.makedirs(self._best_ckeckpoint_name, exist_ok=True)
        self._best_ckeckpoint_name = os.path.join(self._best_ckeckpoint_name, ckeckpoint_name)
    def save_checkpoint(self, model_state_dict, other_state_dict, tag, is_best=False):
        checkpoint_list = list()
        if os.path.exists(self._ckeckpoint_container_file):
            with open(self._ckeckpoint_container_file, 'r') as checkpoint_file:
                for index, line in enumerate(checkpoint_file):
                    if index == 0: continue
                    if line.strip() != '':
                        checkpoint_list.append(line.strip())
        if not is_best: 
            model_save_name = (self._ckeckpoint_name + '_pytorch_state_{}_{}.pth').format(tag, datetime.now().strftime('%Y-%m-%d_%H_%M_%S'))
        else:
            model_save_name = (self._best_ckeckpoint_name + '_pytorch_state_{}_{}.pth').format(tag, datetime.now().strftime('%Y-%m-%d_%H_%M_%S'))
        checkpoint_list.append(model_save_name)

        torch.save(model_state_dict, model_save_name)
        torch.save(other_state_dict, model_save_name.replace('state','others'))

        log.info('model saved: {}.'.format(model_save_name))

        # remove checkpoint older than 5
        if len(checkpoint_list) > self._max_to_keep:
            checkpoint_list_to_delete = checkpoint_list[:-self._max_to_keep]
            checkpoint_list = checkpoint_list[-self._max_to_keep:]
            for model_file in checkpoint_list_to_delete:
                if os.path.isfile(model_file): os.remove(model_file)
                model_file = model_file.replace('state','others')
                if os.path.isfile(model_file):  os.remove(model_file)
        with open(self._ckeckpoint_container_file, 'w') as outfile:
            outfile.write(model_save_name+'\n')
            for line in checkpoint_list:
                outfile.write(line+'\n')

    def restore_from_checkpoint(self, model, step=None):
        checkpoint_filename = None
        if os.path.exists(self._ckeckpoint_container_file):
            with open(self._ckeckpoint_container_file, 'r') as checkpoint_file:
                for _, line in enumerate(checkpoint_file):
                    line = line.strip()
                    if line != '':
                        # get the first one
                        if step is None:
                            checkpoint_filename = line
                            break
                        # get the specified one
                        elif str(step) in line:
                            checkpoint_filename = line
                            break
        if (not os.path.isdir(self._log_dir)) or (checkpoint_filename is None):
            return None

        model.load_state_dict(torch.load(checkpoint_filename, map_location=lambda storage, loc: storage))    

        log.info('model resotred from: {}.'.format(checkpoint_filename))

        return torch.load(checkpoint_filename.replace('state','others'))  

class TimeRecorder(object):
    def __init__(self):
        super(TimeRecorder, self).__init__()
        self._tick_map = dict()
        self._recorder = dict()
        self._use_time = dict()
        self._last_interval = dict()
    # register at the very begining
    # when not use time, call reset_event with your initial value at each start
    def register_event(self, name, tick, use_time=True):
        self._tick_map[name] = tick
        self._last_interval[name] = 0
        self._recorder[name] = 0
        if use_time:
            self._recorder[name] = time.time() 
        self._use_time[name] = use_time
    def reset_event(self, name, criterion=None):
        if self._use_time[name]: self._recorder[name] = time.time()
        elif criterion is not None: self._recorder[name] = criterion
        self._last_interval[name] = 0
    def cancel_event(self, name):
        self._tick_map.pop(name, None)
        self._last_interval.pop(name, None)
        self._recorder.pop(name, None)
        self._use_time.pop(name, None)
    # you can call this to get how log elapsed after you get True from check_for_me
    def how_long_before(self, name):
        return self._last_interval[name]
    def check_for_me(self, name, criterion=None):
        if self._use_time[name]:
            if time.time() - self._recorder[name] > self._tick_map[name]:
                self._last_interval[name] = time.time() - self._recorder[name]
                self._recorder[name] = time.time()
                return True
        elif criterion is not None:
            if criterion - self._recorder[name] > self._tick_map[name]:
                self._last_interval[name] = criterion - self._recorder[name]
                self._recorder[name] = criterion
                return True
        return False
def load_pretrain_file(net, pretrain_file, skip=[]):
    pretrain_state_dict = torch.load(pretrain_file)
    state_dict = net.state_dict()
    keys = list(state_dict.keys())
    for key in keys:
        if any(s in key for s in skip):
            continue
        pretrain_key = key
        #if 'layer0.0.conv.' in key: pretrain_key=key.replace('layer0.0.conv.',  'conv1.' )
        state_dict[key] = pretrain_state_dict[pretrain_key]
        
    net.load_state_dict(state_dict)
    return net

class CriterionSmooth(object):
    def __init__(self):
        super(CriterionSmooth, self).__init__()
        self._history = dict()
        self._factor = dict()
        self._just_add = dict()
    def register_smooth(self, name, factor=0.6):
        self._history[name] = 0.
        self._just_add[name] = False
        if factor < 0: self._just_add[name] = True
        if factor > 1.: factor=1.
        self._factor[name] = 1. - factor
    def push_new_value(self, name, value):
        if self._just_add[name]: self._history[name] = self._history[name] + value
        else: self._history[name] = self._history[name] * (1.-self._factor[name]) + value*self._factor[name]
        return value
    def smooth_value(self, name):
        return self._history[name]


In [6]:
def exp_lr_scheduler(optimizer, epoch, init_lr=0.01, lr_decay_epoch=1):
    lr = init_lr * (0.27**(epoch // lr_decay_epoch))

    if lr < 0.000002: lr = 0.000002
        
    if epoch % lr_decay_epoch == 0:
        log.info('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer
def read_learning_rate(cur_step, num_steps_per_epoch):
    def inner_lr_parser(interval_start, interval_end, lr, dict_in, default_lr, use_epoch_percent, num_steps_per_epoch):
        lr = default_lr * lr
        if use_epoch_percent:
            interval_start = num_steps_per_epoch * interval_start
            interval_end = num_steps_per_epoch * interval_end
        interval_start = int(interval_start)
        interval_end = int(interval_end)
        if (interval_start < interval_end) and (lr > 0):
            dict_in[(interval_start, interval_end)] = lr
            
    lr_map = dict()
    default_lr = initial_learning_rate
    stop_lr = stop_learning_rate
    line_index = -1
    use_epoch_percent = True
    if os.path.exists(LR_FILE_PATH):
        with open(LR_FILE_PATH, 'r') as lr_setting_file:
            for _, line in enumerate(lr_setting_file):
                line = line.strip()
                if (line != '') and (not line.startswith('#')):
                    line_index += 1
                    if line_index == 0:
                        default_lr = float(line.split(':')[-1].strip())
                        continue
                    if line_index == 1:
                        stop_lr = float(line.split(':')[-1].strip())
                        continue
                    if line_index == 2:
                        use_epoch_percent = ('EPOCHES_PERCENT' in (line.split(':')[-1].strip()))
                        continue
                    # this is a list desciption
                    if line.startswith('['):
                        line = [float(s.strip()) for s in line[1:-1].strip().split()]
                        step_interval = (line[1] - line[0])/line[-1]
                        lr_interval = (line[3] - line[2])/line[-1]
                        begin = line[0]
                        lr_begin = line[2]
                        for index in range(int(line[-1])):
                            inner_lr_parser(begin, begin+step_interval, lr_begin, lr_map, default_lr, use_epoch_percent, num_steps_per_epoch)
                            begin += step_interval
                            lr_begin += lr_interval
                    else:
                        interval_start, interval_end, lr = [float(s) for s in line.strip().split()]
                        inner_lr_parser(interval_start, interval_end, lr, lr_map, default_lr, use_epoch_percent, num_steps_per_epoch)
    lr_ret = default_lr
#     print(use_epoch_percent)
    for (start, end), lr in lr_map.items():
        if (cur_step >= start) and (cur_step <= end):
            if (lr < lr_ret):
                lr_ret = lr
    if lr_ret < stop_lr: lr_ret = stop_lr      
    return lr_ret
# _ = read_learning_rate(1, num_steps_per_epoch)
# lr = []
# num_epoches_to_show = 10
# num_point = 100
# for i in [i*num_epoches_to_show*num_steps_per_epoch/num_point for i in range(num_point)]:
#     lr.append(read_learning_rate(i, num_steps_per_epoch))
# plt.plot(lr)
# plt.ylabel('learning rate')
# plt.show()

In [7]:
class LabelMapping(object):
    def __init__(self, catogory_file_path):
        super(LabelMapping, self).__init__()
        self._category_level_csv = catogory_file_path
        self._category_map, self._category_level1_map, self._category_level2_map, self._len_level1, self._len_level2 = self.cvt_csv2tfrecord()
        
        assert (LEVEL1_CLASS == self._len_level1) and (LEVEL2_CLASS == self._len_level2), 'Other two levels are not mapped correctly.'
        self._catogory_weight_map = self.cvt_catogory_weight()
        self._mapping_strings = tf.constant( [ str(key) for key in self._category_map.keys() ] )
        #print(list(self._category_map.keys())[0])
        self._mapping_table = tf.contrib.lookup.index_table_from_tensor(mapping=self._mapping_strings, default_value=0) 
        
        self._level1_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(self._category_level1_map.keys()), list(self._category_level1_map.values()), tf.int64, tf.int64), 0)
        self._level2_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(self._category_level2_map.keys()), list(self._category_level2_map.values()), tf.int64, tf.int64), 0)
        self._weight_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(self._catogory_weight_map.keys()), list(self._catogory_weight_map.values()), tf.int64, tf.float32), 0)

    @property
    def category_map(self):
        return self._category_map
    @property
    def level1_table(self):
        return self._level1_table
    @property
    def level2_table(self):
        return self._level2_table
    @property
    def len_level1(self):
        return self._len_level1
    @property
    def len_level2(self):
        return self._len_level2
    @property
    def mapping_table(self):
        return self._mapping_table
    @property
    def weight_table(self):
        return self._weight_table
    
    def cvt_catogory_weight(self):
        category_weight_map = dict()
        csv = pd.read_csv(CATEGORY_WEIGHT_PATH).values
        for row in csv:  
            category_id, weight = row[0], row[2]
#             if weight > 1.5:
#                 weight = 1.5
            category_weight_map[int(category_id)] = 1.

        return category_weight_map

    def cvt_csv2tfrecord(self):
        level1_map, level2_map = self.create_level_map()
        count = 0
        category_map = dict()
        category_level1_map = dict()
        category_level2_map = dict()
        csv = pd.read_csv(self._category_level_csv).values
        for row in csv:  
            category_id, level1, level2 = row[0], row[1], row[2]
            category_map[category_id] = count
            category_level1_map[int(category_id)] = level1_map[level1]
            category_level2_map[int(category_id)] = level2_map[level2]
            count += 1

        return category_map, category_level1_map, category_level2_map, len(level1_map), len(level2_map)

    def create_level_map(self):
        csv = pd.read_csv(self._category_level_csv).values
        level_list = [list(), list()]
        for row in csv: 
            for level in range(1,3):
                if row[level] not in level_list[level-1]:
                    level_list[level-1].append(row[level])
        return dict(zip(level_list[0], range(len(level_list[0])))), dict(zip(level_list[1], range(len(level_list[1]))))

In [8]:
class CdiscountDataset(Dataset):
    def __init__(self, sess, data_path, file_begin_match, label_mapping, num_examples, num_classes, buffer_size, batch_size, is_training):
        super(CdiscountDataset, self).__init__()
        self._data_file_list = [ os.path.join(data_path, x) for x in os.listdir(data_path) if lambda x: os.path.isfile(x) and file_begin_match in x ]
        self._num_examples = num_examples
        self._tf_sess = sess
        self._num_classes = num_classes
        self._batch_size = batch_size
        self._buffer_size = buffer_size
        self._is_training = is_training
        self._category_map = label_mapping.category_map
        self._level1_table = label_mapping.level1_table
        self._level2_table = label_mapping.level2_table
        self._len_level1 = label_mapping.len_level1
        self._len_level2 = label_mapping.len_level2
        self._mapping_table = label_mapping.mapping_table
    def __len__(self):
        return self._num_examples

    def __getitem__(self, idx):
        #print('read',idx)
        try:
            next_example, next_label, next_level0_label, next_level1_label = self._tf_sess.run(self.get_next())
            #print(next_example, next_label, next_level0_label, next_level1_label)
        except tf.errors.OutOfRangeError:
            pass
        return (torch.from_numpy(next_example), torch.from_numpy(next_label), torch.from_numpy(next_level0_label), torch.from_numpy(next_level1_label))
    @staticmethod
    def image_normalized(image):
        mean = [0.485, 0.456, 0.406 ]
        std  = [0.229, 0.224, 0.225 ]

        image = image.transpose((2,0,1))
        image = image.astype(float)/255.
        
        image[0] = (image[0] - mean[0]) / std[0]
        image[1] = (image[1] - mean[1]) / std[1]
        image[2] = (image[2] - mean[2]) / std[2]

        return image.astype(np.float32)

    @staticmethod
    def fix_center_crop(image, size=(160,160)):

        height, width = image.shape[0:2]
        w,h = size

        x0 = (width  -w)//2
        y0 = (height -h)//2
        x1 = x0 + w
        y1 = y0 + h
        image = image[y0:y1, x0:x1]

        return image
    @staticmethod
    def random_horizontal_flip(image, u=0.5):
        if random.random() < u:
            image = np.flip(image,1)  #np.fliplr(img) ##left-right
        return image
    @staticmethod
    def random_crop(image, size=(160,160), u=0.5):

        height,width=image.shape[0:2]
        w,h = size

        if random.random() < u:
            x0 = np.random.choice(width - w)
            y0 = np.random.choice(height - h)
        else:
            x0 = (width  -w)//2
            y0 = (height -h)//2

        x1 = x0 + w
        y1 = y0 + h
        image = image[y0:y1, x0:x1]

        return image
    @staticmethod
    def random_resize(image, scale_x_limits=[0.9, 1.1], scale_y_limits=[0.9, 1.1], u=0.5):
        if random.random() < u:
            height,width=image.shape[0:2]

            scale_x  = random.uniform(scale_x_limits[0],scale_x_limits[1])
            if scale_y_limits is not None:
                scale_y  = random.uniform(scale_y_limits[0],scale_y_limits[1])
            else:
                scale_y = scale_x

            w = int(scale_x*width )
            h = int(scale_y*height)

            image = imresize(image,(h,w))
        return image

    @staticmethod
    def _preprocess_by_pytorch_train(image):
        image = CdiscountDataset.random_resize(image, scale_x_limits=[0.9,1.1], scale_y_limits=[0.9,1.1], u=0.5)
        # flip  random ---------
        image = CdiscountDataset.random_crop(image, size=(160,160), u=0.5) 
        image = CdiscountDataset.random_horizontal_flip(image, u=0.5)
        return image.astype(np.float32)

    @staticmethod
    def _preprocess_by_pytorch_test(image):
        image  = CdiscountDataset.fix_center_crop(image, size=(160,160))  
        return image.astype(np.float32)
    @staticmethod
    def _preprocess_normalize(image):
        return CdiscountDataset.image_normalized(image)
    
    def _parse_function(self, example_proto):
        features = {'img_raw': tf.FixedLenFeature([], tf.string, default_value=''),
            'product_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
            'category_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))}
                
        parsed_features = tf.parse_single_example(example_proto, features)
        image = tf.image.decode_image(parsed_features["img_raw"])
        raw_label = parsed_features["category_id"]
        if self._is_training:
            image = tf.py_func(CdiscountDataset._preprocess_by_pytorch_train, [image], tf.float32, stateful=False)
            tf.summary.image('final_train_image', tf.expand_dims(image, 0))
        else:
            image = tf.py_func(CdiscountDataset._preprocess_by_pytorch_test, [image], tf.float32, stateful=False)
            tf.summary.image('final_test_image', tf.expand_dims(image, 0))
        image = tf.py_func(CdiscountDataset._preprocess_normalize, [image], tf.float32, stateful=False)
        
        return image, tf.one_hot(self._mapping_table.lookup(tf.as_string(raw_label)), self._num_classes, axis=-1, dtype=tf.int64),\
                tf.one_hot(self._level1_table.lookup(raw_label), self._len_level1, axis=-1, dtype=tf.int64),\
                tf.one_hot(self._level2_table.lookup(raw_label), self._len_level2, axis=-1, dtype=tf.int64)
#         return image, self._mapping_table.lookup(tf.as_string(raw_label)),\
#                 self._level1_table.lookup(raw_label),\
#                 self._level2_table.lookup(raw_label)
    
    def get_next(self):
        #next_example, next_label, next_level0_label, next_level1_label 
        return self._next_iter
    def create_dataset(self):
        self._dataset = tf.data.TFRecordDataset(self._data_file_list, compression_type='ZLIB', buffer_size = 409600)
        parse_func = lambda example : self._parse_function(example)
        self._dataset = self._dataset.map(parse_func, num_parallel_calls=INPUT_THREADS)
        self._dataset = self._dataset.prefetch(self._batch_size * 3)
        self._dataset = self._dataset.shuffle(buffer_size=self._buffer_size)
        # don't batch here, hand this to pytorch dataloader
        #self._dataset = self._dataset.batch(self._batch_size)
        # we don't want to repeat until finish training, instead we stop each one epoch finished
        #self._dataset = self._dataset.repeat(self._num_epochs)
        self._iterator = self._dataset.make_initializable_iterator()
        self._next_iter = self._iterator.get_next()
#             Compute for 100 epochs.
#             for _ in range(100):
#               sess.run(iterator.initializer)
#               while True:
#                 try:
#                   sess.run(next_element)
#                 except tf.errors.OutOfRangeError:
#                   break
#         map(
#             map_func,
#             num_threads=None,
#             output_buffer_size=None,
#             num_parallel_calls=None
#         )
#         Maps map_func across this datset. (deprecated arguments)

#         SOME ARGUMENTS ARE DEPRECATED. They will be removed in a future version. Instructions for updating: Replace num_threads=T with num_parallel_calls=T. Replace output_buffer_size=N with ds.prefetch(N) on the returned dataset.

#         Args:

#         map_func: A function mapping a nested structure of tensors (having shapes and types defined by self.output_shapes and self.output_types) to another nested structure of tensors.
#         num_threads: (Optional.) Deprecated, use num_parallel_calls instead.
#         output_buffer_size: (Optional.) A tf.int64 scalar tf.Tensor, representing the maximum number of processed elements that will be buffered.
#         num_parallel_calls: (Optional.) A tf.int32 scalar tf.Tensor, representing the number elements to process in parallel. If not specified, elements will be processed sequentially.
        return self._iterator.initializer


In [9]:
torch_saver = Saver(LOG_DIR, 'Resnet', 5)

smoother = CriterionSmooth()
smoother.register_smooth('train_correct', -1.)
smoother.register_smooth('train_total', -1.)
smoother.register_smooth('val_correct', -1.)
smoother.register_smooth('val_total', -1.)

timer_holder = TimeRecorder()
timer_holder.register_event('save_time', save_time_interval)
timer_holder.register_event('log_time', log_step_interval, False)
timer_holder.register_event('eval_log_time', log_step_interval, False)

cdiscount_net = torch_resnet.resnet152(pretrained=False, num_classes=NUM_CLASS)
#print(cdiscount_net)
if torch.cuda.is_available():
    cdiscount_net = cdiscount_net.cuda()

#criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(cdiscount_net.parameters(), lr=initial_learning_rate, momentum=momentum, weight_decay=0.0002)

if True:
    cdiscount_net = load_pretrain_file(cdiscount_net, PRETRAINED_MODEL_PATH, ['fc'])
    others_state = None
else:
    others_state = torch_saver.restore_from_checkpoint(cdiscount_net, step=None)
start_iter = 1
start_epoch = 0
global_step = 0
if others_state is not None:
    start_iter  = others_state['iter']
    start_epoch = others_state['epoch']
    global_step  = others_state['global_step']
    optimizer.load_state_dict(others_state['optimizer'])

label_mapping = LabelMapping(CATEGORY_NAME_PATH)
train_set = CdiscountDataset(tf_sess, TRAIN_PATH, 'output_file', label_mapping, TOTAL_EXAMPLES, NUM_CLASS, 10000, BATCH_SIZE, True)
val_set = CdiscountDataset(tf_sess, VAL_PATH, 'output_file', label_mapping, VAL_EXAMPLES, NUM_CLASS, 12000, BATCH_SIZE, False)

merged_summary = tf.summary.merge_all()

train_initializer = train_set.create_dataset()
val_initializer = val_set.create_dataset()
    
tf_sess.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(), tf.tables_initializer()))
#tf_sess.run(train_initializer)
#print(tf_sess.run(train_set.get_next()))
#tf_sess.run(train_set[0])

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, num_workers = 4, drop_last=False)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers = 4, drop_last=False)

# initialize tensorboard_logger
summaries_dir = LOG_DIR + "/tensorboard_logger_{}".format(datetime.now().strftime('%Y-%m-%d_%H_%M_%S'))
configure(summaries_dir, flush_secs=120)
summary_writer = tf.summary.FileWriter(summaries_dir, tf_sess.graph)

#target_onehot = torch.FloatTensor(batch_size, 4)
timer_holder.reset_event('log_time', global_step)
for epoch in range(NUM_EPOCHES):
    if epoch < start_epoch: continue
    log.info('epoch {} of {} start...'.format(epoch, NUM_EPOCHES))
    log.info('start from {} of epoch {}.'.format(start_iter, epoch))
    cdiscount_net.train()
    tf_sess.run(train_initializer)

    for index, data in enumerate(train_loader, start_iter):
        next_example, next_label, next_level0_label, next_level1_label = data
        global_step += 1
            
        if torch.cuda.is_available():
            example_in = Variable(next_example).cuda()
            label_in = Variable(next_label).cuda()
            level0_label_in = Variable(next_level0_label).cuda()
            level1_label_in = Variable(next_level1_label).cuda()
        else:
            example_in = Variable(next_example)
            label_in = Variable(next_label)
            level0_label_in = Variable(next_level0_label)
            level1_label_in = Variable(next_level1_label)
           
        _, label_in = torch.topk(label_in, 1)
        
        target_logits = cdiscount_net(example_in)
        target_softmax = nn.Softmax()(target_logits)
        loss = criterion(target_logits, label_in)
        _, batch_top1 = torch.topk(target_softmax, 1)
        batch_top1 = batch_top1.type(torch.LongTensor)
        if torch.cuda.is_available():
            batch_top1 = batch_top1.cuda()
            
        num_correct = smoother.push_new_value('train_correct', (label_in == batch_top1.view(1, -1)).sum().data[0])
        smoother.push_new_value('train_total', batch_size)
#         target_log_softmax = cdiscount_net(msno_in, artist_in)
#         loss = criterion(target_log_softmax, label_in)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_acc  = num_correct/batch_size
        batch_loss = loss.data[0]
        
        smooth_train_acc = smoother.smooth_value('train_correct')/smoother.smooth_value('train_total')
    
        if timer_holder.check_for_me('log_time', global_step):
            cur_readed_lr = read_learning_rate(global_step, num_steps_per_train_epoch)
            for param_group in optimizer.param_groups:
                param_group['lr'] = cur_readed_lr
            time_passed = timer_holder.how_long_before('log_time')
            log_value('train/batch_loss', batch_loss, global_step)
            log_value('train/sec_per_step', time_passed/log_step_interval, global_step)
            log_value('train/learning_rate', optimizer.param_groups[0]['lr'], global_step)
            log_value('train/batch_acc', batch_acc, global_step)
            log_value('train/smooth_train_acc', smooth_train_acc, global_step)
            # to stdout
            log.info('####### train logging #######')
            log.info('\ttrain/current_step: {}, cur_epoch: {}/{}'.format(global_step, epoch, num_steps_per_train_epoch))
            log.info('\ttrain/current_lr: {}'.format(optimizer.param_groups[0]['lr']))
            log.info('\ttrain/sec_per_step: {:.3f}'.format(time_passed/log_step_interval))
            log.info('\ttrain/batch_loss: {:.3f}'.format(batch_loss))
            log.info('\ttrain/batch_acc: {:.3f}'.format(batch_acc*100))
            log.info('\ttrain/smooth_train_acc: {:.3f}'.format(smooth_train_acc*100))
            summary = tf_sess.run(merged_summary)
            summary_writer.add_summary(summary, global_step)

        if timer_holder.check_for_me('save_time'):
            torch_saver.save_checkpoint(cdiscount_net.state_dict(), {'optimizer': optimizer.state_dict(),
                                                    'iter': index, 'epoch': epoch, 'global_step': global_step }, global_step, is_best=False)
            break
    
    # reset start_iter
    start_iter = 1
    log.info('epoch {} finished.'.format(epoch)) 
    # save model after each epoch
    torch_saver.save_checkpoint(cdiscount_net.state_dict(), {'optimizer': optimizer.state_dict(),
                                                    'iter': 0, 'epoch': epoch, 'global_step': global_step }, global_step, is_best=False)
    timer_holder.reset_event('save_time')
    
    
    # check on validation every epoches            
    cdiscount_net.eval()
    tf_sess.run(val_initializer)
    timer_holder.reset_event('eval_log_time', 1)
    #if False:
    for index, data in enumerate(val_loader, 1):
        next_example, next_label, next_level0_label, next_level1_label = data
            
        if torch.cuda.is_available():
            example_in = Variable(next_example).cuda()
            label_in = Variable(next_label).cuda()
            level0_label_in = Variable(next_level0_label).cuda()
            level1_label_in = Variable(next_level1_label).cuda()
        else:
            example_in = Variable(next_example)
            label_in = Variable(next_label)
            level0_label_in = Variable(next_level0_label)
            level1_label_in = Variable(next_level1_label)
            
        _, label_in = torch.topk(label_in, 1)
           
        _, batch_top1 = torch.topk(nn.Softmax()(cdiscount_net(example_in)), 1)
        batch_top1 = batch_top1.type(torch.LongTensor)
        if torch.cuda.is_available():
            batch_top1 = batch_top1.cuda()
        num_correct = smoother.push_new_value('val_correct', (label_in == batch_top1.view(1, -1)).sum().data[0])
        smoother.push_new_value('val_total', batch_size)        
        
        batch_acc  = num_correct/batch_size

        smooth_validation_acc = smoother.smooth_value('val_correct')/smoother.smooth_value('val_total')

        if timer_holder.check_for_me('eval_log_time', index):
            time_passed = timer_holder.how_long_before('eval_log_time')
            log_value('validation/batch_acc', batch_acc, global_step+index)
            log_value('validation/smooth_validation_acc', smooth_validation_acc, global_step+index)
            log.info('####### validation logging #######')
            log.info('\tvalidation/current_step: {}/{}'.format(index, num_steps_per_val_epoch))
            log.info('\tvalidation/sec_per_step: {:.3f}'.format(time_passed/log_step_interval))
            log.info('\tvalidation/batch_acc: {:.3f}'.format(batch_acc*100))
            log.info('\tvalidation/smooth_validation_acc: {:.3f}'.format(smooth_validation_acc*100))
tf_sess.close()
log.info('Done: {}'.format(datetime.now().strftime('%Y-%m-%d_%H_%M_%S')))

epoch 0 of 12 start...
start from 1 of epoch 0.


KeyboardInterrupt: 